In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import src.model as models
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

/Users/litto/miniconda/envs/TFG/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
VERSION = "3"
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV2"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [3]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
OPTIMIZERS = {
    "Adam": tf.keras.optimizers.legacy.Adam(),
}

In [5]:
LOSS = {
    "BinaryFocalCrossentropy": lambda x, y: tf.keras.losses.BinaryFocalCrossentropy(
        alpha=x, gamma=y
    ),
}

ALPHAS = [0.25, 0.5, 0.75]

GAMMAS = [0.0, 1.0, 2.0, 3.0]

In [6]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

2025-03-01 16:41:13.715755: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-03-01 16:41:13.715782: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-01 16:41:13.715788: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-01 16:41:13.715813: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-01 16:41:13.715826: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
BATCH_SIZE = 64

In [8]:
SUBVERSION = 0
SUB_SUBVERSION = 0

for ALPHA in ALPHAS:
    for GAMMA in GAMMAS:
        LOGS_PATH = os.path.abspath(
            f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )
        MODELS_PATH = os.path.abspath(
            f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )
        REPORTS_PATH = os.path.abspath(
            f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )

        MODEL_COMPLETE_NAME = (
            f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )

        train_datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        )
        train_images = train_datagen.flow_from_dataframe(
            dataframe=train_dataset,
            x_col="file_name",
            y_col="binary_label",
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="binary",
            shuffle=True,
            seed=SEED,
        )

        datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        )
        validation_images = datagen.flow_from_dataframe(
            dataframe=validationtrain_dataset,
            x_col="file_name",
            y_col="binary_label",
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="binary",
            shuffle=True,
            seed=SEED,
        )
        test_images = datagen.flow_from_dataframe(
            dataframe=test_dataset,
            x_col="file_name",
            y_col="binary_label",
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="binary",
            shuffle=False,
            seed=SEED,
        )

        pretrained_model = tf.keras.applications.MobileNetV2(
            weights="imagenet",
            include_top=False,
            input_shape=IMAGE_SHAPE,
        )
        pretrained_model.trainable = False

        model = tf.keras.Sequential(
            [
                pretrained_model,
                tf.keras.layers.GlobalAveragePooling2D(),
                tf.keras.layers.Dense(1, activation="sigmoid"),
            ],
            name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}",
        )

        model.compile(
            optimizer=OPTIMIZERS["RMSprop"],
            loss=LOSS["BinaryFocalCrossentropy"](ALPHA, GAMMA),
            metrics=METRICS,
        )

        history = model.fit(
            train_images,
            epochs=EPOCHS,
            validation_data=validation_images,
            callbacks=[TensorBoard(log_dir=LOGS_PATH)],
        )

        dataframe = pd.DataFrame(history.history)
        history_path = os.path.join(
            LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.csv"
        )
        dataframe.to_csv(history_path, sep=";", index=False)

        os.makedirs(REPORTS_PATH, exist_ok=True)
        accuracy_chart = graphics.create_training_accuracy_chart(
            history_path=history_path,
            model_name=MODEL_COMPLETE_NAME,
        )
        accuracy_chart.write_image(
            f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        loss_chart = graphics.create_training_loss_chart(
            history_path=history_path,
            model_name=MODEL_COMPLETE_NAME,
        )
        loss_chart.write_image(
            f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        os.makedirs(MODELS_PATH, exist_ok=True)
        model.save(MODELS_PATH)

        results = model.evaluate(test_images)
        metric_names = history.model.metrics_names
        evaluation_results = {
            ("test_" + name): value for name, value in zip(metric_names, results)
        }

        evaluation = pd.DataFrame([evaluation_results])
        evaluation.to_csv(
            os.path.join(
                LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.csv"
            ),
            sep=";",
            index=False,
        )

        y_pred_prob = model.predict(test_images)
        y_true = test_images.labels

        fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
        roc_auc = auc(fpr, tpr)

        roc_curve_chart = graphics.create_roc_curve_chart(
            fpr=fpr,
            tpr=tpr,
            roc_auc=roc_auc,
            model_name=MODEL_COMPLETE_NAME,
        )
        roc_curve_chart.write_image(
            f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
        conf_matrix = confusion_matrix(y_true, y_pred_class)
        conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

        confusion_matrix_chart = graphics.create_confusion_matrix_chart(
            conf_matrix=conf_matrix,
            conf_matrix_text=conf_matrix_text,
            model_name=MODEL_COMPLETE_NAME,
        )
        confusion_matrix_chart.write_image(
            f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        y_pred_class = (y_pred_prob > 0.5).astype(int)
        conf_matrix = confusion_matrix(y_true, y_pred_class)
        conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

        confusion_matrix_chart = graphics.create_confusion_matrix_chart(
            conf_matrix=conf_matrix,
            conf_matrix_text=conf_matrix_text,
            model_name=MODEL_COMPLETE_NAME,
        )
        confusion_matrix_chart.write_image(
            f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        SUB_SUBVERSION += 1

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-03-01 16:41:15.651875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.2567 - accuracy: 0.8867 - precision: 0.9090 - recall: 0.9409 - auc: 0.9480

2025-03-01 16:42:08.502655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 65s 229ms/step - loss: 0.2567 - accuracy: 0.8867 - precision: 0.9090 - recall: 0.9409 - auc: 0.9480 - val_loss: 0.1990 - val_accuracy: 0.9188 - val_precision: 0.9353 - val_recall: 0.9425 - val_auc: 0.9738
Epoch 2/10
280/280 [==============================] - 57s 204ms/step - loss: 0.2014 - accuracy: 0.9163 - precision: 0.9404 - recall: 0.9468 - auc: 0.9678 - val_loss: 0.1867 - val_accuracy: 0.9239 - val_precision: 0.9343 - val_recall: 0.9520 - val_auc: 0.9770
Epoch 3/10
280/280 [==============================] - 57s 204ms/step - loss: 0.1877 - accuracy: 0.9208 - precision: 0.9448 - recall: 0.9481 - auc: 0.9722 - val_loss: 0.1794 - val_accuracy: 0.9263 - val_precision: 0.9465 - val_recall: 0.9418 - val_auc: 0.9784
Epoch 4/10
280/280 [==============================] - 59s 210ms/step - loss: 0.1801 - accuracy: 0.9254 - precision: 0.9487 - recall: 0.9505 - auc: 0.9745 - val_loss: 0.1760 - val_accuracy: 0.9288 - val_precision: 0.9448 - val_recall: 

...

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV2/v4/v4.0.0/assets


67/67 [==============================] - 11s 168ms/step - loss: nan - accuracy: 0.3383 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5000


2025-03-01 16:51:16.143155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 11s 160ms/step


ValueError: Input contains NaN.